<a href="https://colab.research.google.com/github/abxda/UP_Python_2025/blob/main/Semana_04_01_Miercoles_UP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Celda 1: Instalación de Librerías Esenciales
# -------------------------------------------
# Usamos 'pip install' para añadir a nuestro entorno Python las herramientas (librerías)
# que necesitaremos para este proyecto. El comando '--quiet' reduce la cantidad de mensajes
# durante la instalación.

# Librerías a instalar:
# - duckdb: Para nuestra base de datos analítica rápida y basada en archivos.
# - geopandas: Para leer, escribir y manipular datos geoespaciales (mapas).
# - fsspec: Una dependencia de geopandas para trabajar con diferentes sistemas de archivos.
# - matplotlib: Aunque no graficaremos explícitamente, es una dependencia común de geopandas.
# - tqdm: Para mostrar barras de progreso visuales durante tareas largas.
# - requests: Para descargar archivos de internet (datos del INEGI).
# - pyarrow: Para que geopandas y duckdb puedan trabajar eficientemente con el formato GeoParquet.
# - folium: Para crear mapas interactivos.
# - shapely: Para operaciones geométricas (necesaria con GeoPandas y para WKB/WKT).

!pip install duckdb geopandas fsspec matplotlib tqdm requests pyarrow folium shapely --quiet

In [ ]:
# Celda 2: Importación de Librerías
# ----------------------------------
# Aquí cargamos las librerías que instalamos o que vienen con Python
# para poder usar sus funciones en nuestro código.

# --- Para interactuar con el sistema de archivos y utilidades generales ---
import os  # Funciones para interactuar con el sistema operativo (rutas, carpetas)
import shutil  # Operaciones de alto nivel con archivos y carpetas (ej. borrar carpetas)
import time  # Funciones relacionadas con el tiempo (ej. pausas, aunque no la usaremos activamente)
from zipfile import ZipFile  # Para trabajar con archivos comprimidos .zip
import importlib.metadata # Para checar si pyarrow está instalado

# --- Para descargas web y barras de progreso ---
import requests  # Para hacer solicitudes HTTP (descargar archivos de internet)
from tqdm import tqdm  # Para mostrar barras de progreso visuales

# --- Para análisis de datos y geoespacial ---
import duckdb  # Para la base de datos analítica DuckDB
import geopandas as gpd  # Para trabajar con datos geoespaciales (Shapefiles, GeoParquet)
from shapely import wkb, wkt # Para convertir entre WKB/WKT y objetos de geometría
from shapely import errors as shapely_errors # Para manejo de errores de shapely

# --- Para visualización ---
import folium # Para mapas interactivos

# (Nota: pyarrow se usa internamente por geopandas y duckdb para Parquet,
# no necesitamos importarlo directamente aquí para las operaciones que haremos,
# pero su instalación es crucial.)

print("Librerías importadas exitosamente.")

In [ ]:
# Celda 3: Función de Descarga Programática
# -----------------------------------------
# Esta función se encarga de descargar un archivo desde una URL
# y guardarlo en un directorio específico. Incluye una barra de
# progreso y evita descargar archivos que ya existen.

def download(url, directory):
    """
    Descarga un archivo desde la URL especificada y lo guarda en 'directory'.

    Si el archivo ya existe en 'directory', no realiza la descarga de nuevo.
    Muestra una barra de progreso durante la descarga.
    Detiene el script si ocurre un error HTTP (ej. archivo no encontrado).

    Args:
        url (str): La URL completa del archivo a descargar.
        directory (str): La ruta de la carpeta donde se guardará el archivo.
    """
    # Extraer el nombre del archivo de la URL
    filename = url.split('/')[-1]
    filepath = os.path.join(directory, filename)

    if os.path.exists(filepath):
        print(f"El archivo '{filename}' ya existe en '{directory}'. No se descarga de nuevo.")
        return

    print(f"Descargando '{filename}' de '{url}'...")
    try:
        response = requests.get(url, stream=True, timeout=30)
        response.raise_for_status()
        total_size_in_bytes = int(response.headers.get('content-length', 0))
        progress_bar_params = {
            'desc': filename,
            'total': total_size_in_bytes,
            'unit': 'B',
            'unit_scale': True,
            'unit_divisor': 1024,
            'ncols': 80
        }
        if total_size_in_bytes == 0:
            progress_bar_params.pop('total', None)

        with open(filepath, 'wb') as f, tqdm(**progress_bar_params) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))
        print(f"\nDescarga de '{filename}' completada y guardada en '{filepath}'.\n")
    except requests.exceptions.HTTPError as http_err:
        print(f"Error HTTP durante la descarga de '{filename}': {http_err}")
        if os.path.exists(filepath): os.remove(filepath)
        raise
    except Exception as e:
        print(f"Ocurrió un error inesperado durante la descarga de '{filename}': {e}")
        if os.path.exists(filepath): os.remove(filepath)
        raise

In [ ]:
# Celda 4: Función para Extraer Componentes de Shapefiles desde Archivos ZIP
# -----------------------------------------------------------------------
def extract_shapefile(list_of_zip_filenames, zip_file_directory, target_shp_output_dir, shape_type_suffix):
    """
    Extrae los componentes de un Shapefile (.shp, .shx, .dbf, .prj, .cpg)
    desde los archivos ZIP especificados y los guarda en 'target_shp_output_dir'.
    """
    os.makedirs(target_shp_output_dir, exist_ok=True)
    for zip_filename in list_of_zip_filenames:
        if '_' in zip_filename:
            state_code = zip_filename.split('_')[0]
        else:
            potential_code = zip_filename[:2]
            if potential_code.isdigit():
                state_code = potential_code
            else:
                state_code = os.path.splitext(zip_filename)[0]
                print(f"  Advertencia: No se pudo determinar el código de estado para {zip_filename}. Usando '{state_code}' como base.")

        full_zip_path = os.path.join(zip_file_directory, zip_filename)
        if not os.path.exists(full_zip_path):
            print(f"¡ERROR! Archivo ZIP no encontrado: {full_zip_path}. Saltando este archivo.")
            continue

        expected_output_basenames = [
            f'{state_code}{shape_type_suffix}.shp', f'{state_code}{shape_type_suffix}.cpg',
            f'{state_code}{shape_type_suffix}.dbf', f'{state_code}{shape_type_suffix}.prj',
            f'{state_code}{shape_type_suffix}.shx'
        ]
        expected_output_filepaths = [os.path.join(target_shp_output_dir, basename) for basename in expected_output_basenames]
        paths_in_zip = [f'conjunto_de_datos/{basename}' for basename in expected_output_basenames]

        if all(os.path.exists(filepath) for filepath in expected_output_filepaths):
            print(f"Todos los archivos Shapefile para '{state_code}{shape_type_suffix}' ya existen en '{target_shp_output_dir}'. No se extraen de nuevo.")
            continue

        print(f"\nProcesando extracción de Shapefiles para '{state_code}{shape_type_suffix}' de '{zip_filename}'...")
        try:
            with ZipFile(full_zip_path, 'r') as zip_ref:
                for path_in_zip_archive, target_output_filepath, output_basename in zip(paths_in_zip, expected_output_filepaths, expected_output_basenames):
                    if os.path.exists(target_output_filepath):
                        print(f"  Archivo '{output_basename}' ya existe. Saltando.")
                        continue
                    try:
                        with zip_ref.open(path_in_zip_archive) as source_file, open(target_output_filepath, 'wb') as target_file:
                            shutil.copyfileobj(source_file, target_file)
                        print(f"  Extraído: '{output_basename}' a '{target_output_filepath}'")
                    except KeyError:
                        if output_basename.endswith('.cpg'):
                            try:
                                with open(target_output_filepath, 'w') as cpg_file:
                                    cpg_file.write("ISO-8859-1") # O UTF-8 según necesidad
                                print(f"  ADVERTENCIA: '{output_basename}' no encontrado. Se creó '{target_output_filepath}' con codificación por defecto.")
                            except Exception as e_cpg:
                                print(f"  ERROR al crear archivo .cpg por defecto '{target_output_filepath}': {e_cpg}")
                        else:
                            print(f"  ¡ERROR CRÍTICO! Archivo esencial '{path_in_zip_archive}' no encontrado en '{zip_filename}'.")
                    except Exception as e_extract:
                        print(f"  ERROR inesperado al extraer '{path_in_zip_archive}': {e_extract}")
            print(f"Extracción para '{state_code}{shape_type_suffix}' completada.")
        except FileNotFoundError:
             print(f"¡ERROR! Archivo ZIP no encontrado en la ruta: {full_zip_path}")
        except Exception as e_zip:
            print(f"Ocurrió un error general al procesar '{zip_filename}': {e_zip}")

In [ ]:
# Celda 5: Definición de Variables de Configuración y Organización de Carpetas
# ---------------------------------------------------------------------------

# --- Lista de todos los estados de México (comentada para referencia) ---
# estados_geo_todos = [
#     "01_aguascalientes.zip", "02_bajacalifornia.zip", "03_bajacaliforniasur.zip",
#     "04_campeche.zip", "05_coahuiladezaragoza.zip", "06_colima.zip",
#     "07_chiapas.zip", "08_chihuahua.zip", "09_ciudaddemexico.zip",
#     "10_durango.zip", "11_guanajuato.zip", "12_guerrero.zip",
#     "13_hidalgo.zip", "14_jalisco.zip", "15_mexico.zip",
#     "16_michoacandeocampo.zip", "17_morelos.zip", "18_nayarit.zip",
#     "19_nuevoleon.zip", "20_oaxaca.zip", "21_puebla.zip",
#     "22_queretaro.zip", "23_quintanaroo.zip", "24_sanluispotosi.zip",
#     "25_sinaloa.zip", "26_sonora.zip", "27_tabasco.zip",
#     "28_tamaulipas.zip", "29_tlaxcala.zip", "30_veracruzignaciodelallave.zip",
#     "31_yucatan.zip", "32_zacatecas.zip"
# ]
# estados_num_todos = [
#     1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
#     20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32
# ]

# --- 1. Variables de Configuración del Estado a Procesar (Aguascalientes) ---
NOMBRE_ESTADO = "Aguascalientes"
CODIGO_ESTADO_NUM = 1
CODIGO_ESTADO_STR = f"{CODIGO_ESTADO_NUM:02d}" # Será "01"

# Nombre del archivo ZIP del Marco Geoestadístico para Aguascalientes
ESTADO_GEO_ZIP_BASENAME = f"{CODIGO_ESTADO_STR}_aguascalientes" # "01_aguascalientes"
ESTADO_GEO_ZIP_FILENAME = f"{ESTADO_GEO_ZIP_BASENAME}.zip" # "01_aguascalientes.zip"

TIPO_SHAPEFILE_MANZANAS = "m"

print(f"--- Configuración para el Estado: {NOMBRE_ESTADO} (Código: {CODIGO_ESTADO_STR}) ---")
print(f"Archivo ZIP del Marco Geoestadístico a buscar: {ESTADO_GEO_ZIP_FILENAME}")
print(f"Tipo de Shapefile a extraer: Manzanas (sufijo '{TIPO_SHAPEFILE_MANZANAS}')")

# --- 2. Definición de Rutas de Carpetas ---
DIR_BASE_INEGI = "./inegi_data_ags" # Directorio específico para Aguascalientes
DIR_CENSO_CSV_BASE = os.path.join(DIR_BASE_INEGI, "censo_poblacion_vivienda_csv")
DIR_CENSO_CSV_DESCARGAS = os.path.join(DIR_CENSO_CSV_BASE, "descargas_zip")
DIR_CENSO_CSV_EXTRAIDOS = os.path.join(DIR_CENSO_CSV_BASE, "csv_extraidos")
DIR_MARCO_GEO_BASE = os.path.join(DIR_BASE_INEGI, "marco_geoestadistico_shp")
DIR_MARCO_GEO_DESCARGAS_ZIP = os.path.join(DIR_MARCO_GEO_BASE, "descargas_zip")
DIR_MARCO_GEO_SHP_EXTRAIDOS = os.path.join(DIR_MARCO_GEO_BASE, "shp_extraidos")
DIR_SHP_MANZANAS_EXTRAIDOS = os.path.join(DIR_MARCO_GEO_SHP_EXTRAIDOS, TIPO_SHAPEFILE_MANZANAS)
DB_FILENAME = f"inegi_analisis_{CODIGO_ESTADO_STR}.duckdb" # inegi_analisis_01.duckdb
DB_FILE_PATH = os.path.join(DIR_BASE_INEGI, DB_FILENAME)

print(f"\n--- Rutas de Carpetas Definidas ---")
print(f"Directorio Base del Proyecto: {os.path.abspath(DIR_BASE_INEGI)}")
print(f"  CSVs (Descargas ZIP): {os.path.abspath(DIR_CENSO_CSV_DESCARGAS)}")
print(f"  CSVs (Extraídos): {os.path.abspath(DIR_CENSO_CSV_EXTRAIDOS)}")
print(f"  Shapefiles (Descargas ZIP): {os.path.abspath(DIR_MARCO_GEO_DESCARGAS_ZIP)}")
print(f"  Shapefiles Manzanas (Extraídos): {os.path.abspath(DIR_SHP_MANZANAS_EXTRAIDOS)}")
print(f"  Archivo Base de Datos DuckDB: {os.path.abspath(DB_FILE_PATH)}")

# --- 3. Limpieza Opcional de Directorios Existentes ---
LIMPIAR_DIRECTORIOS_ANTES_DE_EJECUTAR = True
if LIMPIAR_DIRECTORIOS_ANTES_DE_EJECUTAR:
    print("\n--- Limpieza de Directorios (si existen) ---")
    directorios_a_limpiar_base = [DIR_CENSO_CSV_BASE, DIR_MARCO_GEO_BASE]
    if os.path.exists(DB_FILE_PATH):
        print(f"Eliminando archivo de base de datos existente: {DB_FILE_PATH}")
        try: os.remove(DB_FILE_PATH)
        except Exception as e: print(f"  No se pudo eliminar {DB_FILE_PATH}: {e}")
    for directorio_base in directorios_a_limpiar_base:
        if os.path.exists(directorio_base):
            print(f"Eliminando directorio base existente y su contenido: {directorio_base}")
            try: shutil.rmtree(directorio_base)
            except Exception as e: print(f"  No se pudo eliminar {directorio_base}: {e}")
        else:
            print(f"Directorio base {directorio_base} no existe, no se necesita limpieza.")
else:
    print("\n--- Limpieza de Directorios Omitida ---")

# --- 4. Creación de la Estructura de Carpetas Necesaria ---
print("\n--- Creación de Estructura de Carpetas Necesarias ---")
directorios_a_crear = [
    DIR_BASE_INEGI, DIR_CENSO_CSV_DESCARGAS, DIR_CENSO_CSV_EXTRAIDOS,
    DIR_MARCO_GEO_DESCARGAS_ZIP, DIR_SHP_MANZANAS_EXTRAIDOS
]
for directorio in directorios_a_crear:
    try:
        os.makedirs(directorio, exist_ok=True)
        print(f"Directorio asegurado/creado: {directorio}")
    except Exception as e:
        print(f"Error al crear directorio {directorio}: {e}.")
        raise
print("\n--- Configuración de variables y carpetas completada. ---")

In [ ]:
# Celda 6: Conexión a DuckDB e Instalación/Carga de la Extensión Espacial
# --------------------------------------------------------------------
print(f"--- Configuración de DuckDB ---")
if os.path.exists(DB_FILE_PATH):
    print(f"Archivo de base de datos ya existe en: {DB_FILE_PATH}")
else:
    print(f"Archivo de base de datos no encontrado. Se creará en: {DB_FILE_PATH}")

print(f"Conectando a la base de datos DuckDB en: '{DB_FILE_PATH}'...")
try:
    con = duckdb.connect(database=DB_FILE_PATH, read_only=False)
    print("Conexión a DuckDB establecida exitosamente.")
except Exception as e:
    print(f"¡ERROR CRÍTICO al conectar con DuckDB!: {e}")
    raise

print("\nInstalando (si es necesario) y cargando la extensión 'spatial' de DuckDB...")
try:
    con.execute("INSTALL spatial;")
    con.execute("LOAD spatial;")
    print("Extensión 'spatial' de DuckDB instalada y cargada correctamente.")
except duckdb.IOException as e:
    print(f"¡ERROR DE ENTRADA/SALIDA (IOException) al instalar/cargar la extensión 'spatial'!: {e}")
    print("Verifica tu conexión a internet y/o los permisos del directorio de extensiones de DuckDB.")
    raise
except Exception as e:
    print(f"¡ERROR GENERAL al instalar/cargar la extensión 'spatial'!: {e}")
    raise
print("\n--- Configuración de DuckDB completada. ---")

In [ ]:
# Celda 7: Descarga y Extracción del Archivo CSV con Datos Censales
# -----------------------------------------------------------------
print(f"--- Iniciando Proceso de Descarga y Extracción de CSV para el Estado: {CODIGO_ESTADO_STR} ---")

URL_BASE_CENSO_CSV_INEGI = "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/datosabiertos/ageb_manzana/"
NOMBRE_ZIP_CENSO = f"ageb_mza_urbana_{CODIGO_ESTADO_STR}_cpv2020_csv.zip" # ageb_mza_urbana_01_cpv2020_csv.zip
URL_COMPLETA_CENSO_CSV_ZIP = f"{URL_BASE_CENSO_CSV_INEGI}{NOMBRE_ZIP_CENSO}"
RUTA_DESCARGA_ZIP_CENSO = os.path.join(DIR_CENSO_CSV_DESCARGAS, NOMBRE_ZIP_CENSO)

CARPETA_RAIZ_EN_ZIP_CENSO = f"ageb_mza_urbana_{CODIGO_ESTADO_STR}_cpv2020"
SUBPATH_CSV_EN_ZIP_CENSO = "conjunto_de_datos"
# El nombre del CSV *dentro* del ZIP es usualmente "conjunto_de_datos_ageb_urbana_[CODIGO_ESTADO]_cpv2020.csv"
# Sin embargo, algunos estados pueden tener una ligera variación, ej. solo "conjunto_de_datos_ageb_mza_urbana_[CODIGO_ESTADO]_cpv2020.csv"
# Es crucial verificar el contenido del ZIP si la extracción falla.
# Intentaremos con el nombre más común y luego el alternativo si falla.

NOMBRE_CSV_ORIGINAL_EN_ZIP_OPCION1 = f"conjunto_de_datos_ageb_urbana_{CODIGO_ESTADO_STR}_cpv2020.csv"
PATH_COMPLETO_CSV_DENTRO_DEL_ZIP_OPCION1 = os.path.join(CARPETA_RAIZ_EN_ZIP_CENSO, SUBPATH_CSV_EN_ZIP_CENSO, NOMBRE_CSV_ORIGINAL_EN_ZIP_OPCION1)

# Alternativa (a veces encontrada, ej. para CDMX el nombre es un poco diferente, pero para Aguascalientes el de arriba es el correcto)
# NOMBRE_CSV_ORIGINAL_EN_ZIP_OPCION2 = f"conjunto_de_datos_ageb_mza_urbana_{CODIGO_ESTADO_STR}_cpv2020.csv"
# PATH_COMPLETO_CSV_DENTRO_DEL_ZIP_OPCION2 = os.path.join(CARPETA_RAIZ_EN_ZIP_CENSO, SUBPATH_CSV_EN_ZIP_CENSO, NOMBRE_CSV_ORIGINAL_EN_ZIP_OPCION2)


NOMBRE_CSV_FINAL_EXTRAIDO = f"censo_manzanas_urbanas_{CODIGO_ESTADO_STR}_cpv2020.csv"
RUTA_FINAL_CSV_EXTRAIDO = os.path.join(DIR_CENSO_CSV_EXTRAIDOS, NOMBRE_CSV_FINAL_EXTRAIDO)

print(f"URL para descarga del ZIP del censo: {URL_COMPLETA_CENSO_CSV_ZIP}")
print(f"Ruta de descarga del ZIP: {RUTA_DESCARGA_ZIP_CENSO}")
print(f"Ruta (opción 1) del CSV dentro del ZIP: {PATH_COMPLETO_CSV_DENTRO_DEL_ZIP_OPCION1}")
print(f"Ruta final del CSV extraído: {RUTA_FINAL_CSV_EXTRAIDO}")

if not os.path.exists(RUTA_DESCARGA_ZIP_CENSO):
    print(f"\nDescargando '{NOMBRE_ZIP_CENSO}'...")
    try:
        download(URL_COMPLETA_CENSO_CSV_ZIP, DIR_CENSO_CSV_DESCARGAS)
    except Exception as e:
        print(f"  La descarga del ZIP del censo falló: {e}. No se puede continuar.")
        raise
else:
    print(f"\nEl archivo ZIP '{NOMBRE_ZIP_CENSO}' ya existe. No se descarga de nuevo.")

if not os.path.exists(RUTA_FINAL_CSV_EXTRAIDO):
    print(f"\nExtrayendo CSV de '{NOMBRE_ZIP_CENSO}'...")
    if not os.path.exists(RUTA_DESCARGA_ZIP_CENSO):
        print(f"  ¡ERROR! No se encontró '{RUTA_DESCARGA_ZIP_CENSO}'. No se puede extraer.")
        raise FileNotFoundError(f"ZIP del censo no encontrado: {RUTA_DESCARGA_ZIP_CENSO}")
    else:
        extracted_successfully = False
        try:
            with ZipFile(RUTA_DESCARGA_ZIP_CENSO, 'r') as zip_ref:
                # Intentar Opción 1
                print(f"  Intentando extraer: {PATH_COMPLETO_CSV_DENTRO_DEL_ZIP_OPCION1}")
                zip_ref.open(PATH_COMPLETO_CSV_DENTRO_DEL_ZIP_OPCION1) # Probar si existe
                with zip_ref.open(PATH_COMPLETO_CSV_DENTRO_DEL_ZIP_OPCION1) as source_csv, \
                     open(RUTA_FINAL_CSV_EXTRAIDO, 'wb') as target_csv:
                    shutil.copyfileobj(source_csv, target_csv)
                print(f"  Archivo CSV extraído exitosamente a: '{RUTA_FINAL_CSV_EXTRAIDO}' usando Opción 1.")
                extracted_successfully = True
        except KeyError:
            print(f"  ¡ERROR! No se encontró '{PATH_COMPLETO_CSV_DENTRO_DEL_ZIP_OPCION1}' en el ZIP.")
            print(f"  Verifica la estructura interna del ZIP del INEGI para el estado {CODIGO_ESTADO_STR}.")
            # Podrías añadir aquí el intento con PATH_COMPLETO_CSV_DENTRO_DEL_ZIP_OPCION2 si fuera necesario
            # pero para Aguascalientes, la opción 1 es la correcta según la URL de ejemplo.
        except Exception as e:
            print(f"  ¡ERROR! Ocurrió un error inesperado durante la extracción: {e}")
            if os.path.exists(RUTA_FINAL_CSV_EXTRAIDO): os.remove(RUTA_FINAL_CSV_EXTRAIDO) # Limpiar si se creó parcialmente
            raise
        if not extracted_successfully:
             print(f"  Fallo en la extracción del CSV. El archivo {RUTA_FINAL_CSV_EXTRAIDO} no se creó.")
             # Considerar detenerse si la extracción falla
             raise RuntimeError(f"No se pudo extraer el archivo CSV necesario del ZIP {NOMBRE_ZIP_CENSO}")

else:
    print(f"\nEl archivo CSV '{NOMBRE_CSV_FINAL_EXTRAIDO}' ya existe. No se extrae de nuevo.")

print("\n--- Proceso de Descarga y Extracción de CSV completado (o intentado). ---")
if os.path.exists(RUTA_FINAL_CSV_EXTRAIDO):
    print(f"Confirmado: El archivo CSV está listo en: {RUTA_FINAL_CSV_EXTRAIDO}")
else:
    print(f"ADVERTENCIA: El archivo CSV final NO se encuentra en: {RUTA_FINAL_CSV_EXTRAIDO}.")
    raise FileNotFoundError(f"Archivo CSV final no encontrado: {RUTA_FINAL_CSV_EXTRAIDO}")

In [ ]:
# Celda 8: Cargar el CSV de Datos Censales a una Tabla en DuckDB
# -------------------------------------------------------------
print(f"--- Cargando Datos del CSV a la Tabla 'censo_all' en DuckDB ---")
if not os.path.exists(RUTA_FINAL_CSV_EXTRAIDO):
    print(f"¡ERROR CRÍTICO! No se encontró '{RUTA_FINAL_CSV_EXTRAIDO}'. La tabla 'censo_all' no se puede crear.")
    raise FileNotFoundError(f"Archivo CSV del censo no encontrado: {RUTA_FINAL_CSV_EXTRAIDO}")
else:
    print(f"Archivo CSV encontrado: '{RUTA_FINAL_CSV_EXTRAIDO}'. Procediendo a crear la tabla 'censo_all'.")
    try:
        con.execute("DROP TABLE IF EXISTS censo_all;")
        print("Tabla 'censo_all' eliminada si existía previamente.")
        # Asegúrate de que la columna 'MZA' exista. El INEGI a veces usa 'MANZANA'.
        # Verifica el encabezado del CSV si hay errores aquí.
        # Para Aguascalientes, el CSV de "ageb_mza_urbana_01_cpv2020" usa MZA.
        sql_create_table_from_csv = f"""
        CREATE TABLE censo_all AS
        SELECT *
        FROM read_csv_auto('{RUTA_FINAL_CSV_EXTRAIDO}',
                           NULLSTR=['N/A', 'N/D', '*', ''],
                           SAMPLE_SIZE=-1,
                           HEADER=TRUE
                          )
        WHERE MZA != '000';
        """
        print("\nEjecutando la creación de la tabla 'censo_all' desde el CSV...")
        con.execute("BEGIN TRANSACTION;")
        con.execute(sql_create_table_from_csv)
        con.execute("COMMIT;")
        print("¡Tabla 'censo_all' creada y poblada exitosamente!")

        print("\nVerificando la tabla 'censo_all'...")
        num_filas = con.execute("SELECT COUNT(*) FROM censo_all;").fetchone()[0]
        print(f"  Número total de filas en 'censo_all' (con MZA != '000'): {num_filas}")
        if num_filas == 0:
            print("  ADVERTENCIA: La tabla 'censo_all' está vacía. Verifica el CSV y el filtro MZA.")

        print("\n  Estructura (primeras 5 columnas) y primeras 3 filas de 'censo_all':")
        preview_df = con.execute("SELECT ENTIDAD, NOM_ENT, MUN, NOM_MUN, LOC, NOM_LOC, AGEB, MZA, POBTOT FROM censo_all LIMIT 3;").fetchdf()
        if preview_df.empty and num_filas > 0:
             print("  No se pudieron obtener las columnas de vista previa, pero la tabla tiene filas.")
        elif not preview_df.empty:
            print(preview_df)
        else:
            print("  La tabla 'censo_all' parece estar vacía.")
    except Exception as e:
        print(f"¡ERROR GENERAL al crear la tabla 'censo_all' desde CSV!: {e}")
        try: con.execute("ROLLBACK;")
        except: pass
        raise
print("\n--- Proceso de carga de CSV a DuckDB completado (o intentado). ---")

In [ ]:
# Celda 8.1: Ejemplos de Consultas SQL Básicas sobre la Tabla 'censo_all'
# (Esta celda es opcional, pero útil para verificar)
# Se omite para brevedad en la solicitud principal, pero se puede reactivar.
print(f"--- (Opcional) Ejemplos de Consultas SQL sobre la tabla 'censo_all' ---")
try:
    num_filas_censo_all_check = con.execute("SELECT COUNT(*) FROM censo_all;").fetchone()[0]
    if num_filas_censo_all_check > 0:
        print(f"La tabla 'censo_all' contiene {num_filas_censo_all_check} filas.")
        df_ej_nom_mun = con.execute("SELECT DISTINCT NOM_MUN FROM censo_all LIMIT 5;").fetchdf()
        print("\nPrimeros 5 municipios/alcaldías encontrados:")
        print(df_ej_nom_mun)

        df_ej_pobtot_ags = con.execute("""
            SELECT NOM_MUN, SUM(TRY_CAST(POBTOT AS BIGINT)) AS PoblacionTotal
            FROM censo_all
            GROUP BY NOM_MUN
            ORDER BY PoblacionTotal DESC;
        """).fetchdf()
        print("\nPoblación total por municipio:")
        print(df_ej_pobtot_ags)
    else:
        print("La tabla 'censo_all' está vacía, no se pueden ejecutar ejemplos de consulta.")
except Exception as e:
    print(f"Error ejecutando consultas de ejemplo: {e}")
print(f"--- Fin de ejemplos SQL opcionales ---")

In [ ]:
# Celda 9: Descarga y Extracción del Shapefile de Manzanas
# --------------------------------------------------------
print(f"--- Iniciando Proceso de Descarga y Extracción de Shapefiles de Manzanas para: {CODIGO_ESTADO_STR} ---")

URL_BASE_MARCO_GEO_INEGI = "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/"
# ESTADO_GEO_ZIP_FILENAME ya está definido como "01_aguascalientes.zip"
URL_COMPLETA_MARCO_GEO_ZIP = f"{URL_BASE_MARCO_GEO_INEGI}{ESTADO_GEO_ZIP_FILENAME}"
RUTA_DESCARGA_ZIP_MARCO_GEO = os.path.join(DIR_MARCO_GEO_DESCARGAS_ZIP, ESTADO_GEO_ZIP_FILENAME)

print(f"URL para descarga del ZIP del Marco Geoestadístico: {URL_COMPLETA_MARCO_GEO_ZIP}")
print(f"Ruta de descarga del ZIP: {RUTA_DESCARGA_ZIP_MARCO_GEO}")
print(f"Directorio de salida para Shapefiles de manzanas extraídos: {DIR_SHP_MANZANAS_EXTRAIDOS}")

if not os.path.exists(RUTA_DESCARGA_ZIP_MARCO_GEO):
    print(f"\nDescargando '{ESTADO_GEO_ZIP_FILENAME}' (Marco Geoestadístico)...")
    try:
        download(URL_COMPLETA_MARCO_GEO_ZIP, DIR_MARCO_GEO_DESCARGAS_ZIP)
    except Exception as e:
        print(f"  La descarga del ZIP del Marco Geoestadístico falló: {e}")
        raise
else:
    print(f"\nEl archivo ZIP '{ESTADO_GEO_ZIP_FILENAME}' ya existe. No se descarga de nuevo.")

print(f"\nExtrayendo componentes del Shapefile de tipo '{TIPO_SHAPEFILE_MANZANAS}' (Manzanas)...")
if not os.path.exists(RUTA_DESCARGA_ZIP_MARCO_GEO):
    print(f"  ¡ERROR CRÍTICO! No se encontró '{RUTA_DESCARGA_ZIP_MARCO_GEO}'. No se pueden extraer Shapefiles.")
    raise FileNotFoundError(f"ZIP del Marco Geoestadístico no encontrado: {RUTA_DESCARGA_ZIP_MARCO_GEO}")
else:
    try:
        extract_shapefile(
            list_of_zip_filenames=[ESTADO_GEO_ZIP_FILENAME],
            zip_file_directory=DIR_MARCO_GEO_DESCARGAS_ZIP,
            target_shp_output_dir=DIR_SHP_MANZANAS_EXTRAIDOS,
            shape_type_suffix=TIPO_SHAPEFILE_MANZANAS
        )
        print(f"Proceso de extracción de Shapefiles de manzanas intentado.")
    except Exception as e:
        print(f"  Ocurrió un error inesperado durante la llamada a extract_shapefile: {e}")
        raise

print("\n--- Proceso de Descarga y Extracción de Shapefiles de Manzanas completado (o intentado). ---")
archivo_shp_esperado = os.path.join(DIR_SHP_MANZANAS_EXTRAIDOS, f"{CODIGO_ESTADO_STR}{TIPO_SHAPEFILE_MANZANAS}.shp")
if os.path.exists(archivo_shp_esperado):
    print(f"Confirmado: '{os.path.basename(archivo_shp_esperado)}' existe en '{DIR_SHP_MANZANAS_EXTRAIDOS}'.")
else:
    print(f"ADVERTENCIA: '{os.path.basename(archivo_shp_esperado)}' NO se encuentra.")
    raise FileNotFoundError(f"Archivo SHP principal no encontrado: {archivo_shp_esperado}")

In [ ]:
# Celda 10: Conversión de Shapefile a Formato GeoParquet
# ----------------------------------------------------
print(f"--- Iniciando Proceso de Conversión de Shapefile a GeoParquet para: {CODIGO_ESTADO_STR} ---")

NOMBRE_SHP_BASE = f"{CODIGO_ESTADO_STR}{TIPO_SHAPEFILE_MANZANAS}" # ej: "01m"
RUTA_SHP_ENTRADA = os.path.join(DIR_SHP_MANZANAS_EXTRAIDOS, f"{NOMBRE_SHP_BASE}.shp")
RUTA_GEOPARQUET_SALIDA = os.path.join(DIR_SHP_MANZANAS_EXTRAIDOS, f"{NOMBRE_SHP_BASE}.geoparquet")

print(f"Archivo Shapefile de entrada: {RUTA_SHP_ENTRADA}")
print(f"Archivo GeoParquet de salida: {RUTA_GEOPARQUET_SALIDA}")

if not os.path.exists(RUTA_SHP_ENTRADA):
    print(f"  ¡ERROR CRÍTICO! No se encontró '{RUTA_SHP_ENTRADA}'. No se puede convertir.")
    raise FileNotFoundError(f"Archivo Shapefile de entrada no encontrado: {RUTA_SHP_ENTRADA}")
elif os.path.exists(RUTA_GEOPARQUET_SALIDA):
    print(f"\nEl GeoParquet '{os.path.basename(RUTA_GEOPARQUET_SALIDA)}' ya existe. No se convierte de nuevo.")
else:
    print(f"\nIniciando conversión de '{os.path.basename(RUTA_SHP_ENTRADA)}' a GeoParquet...")
    try:
        print(f"  Leyendo Shapefile: '{RUTA_SHP_ENTRADA}'...")
        gdf = gpd.read_file(RUTA_SHP_ENTRADA, encoding='ISO-8859-1') # O 'UTF-8' si es necesario
        print(f"    Shapefile leído. Contiene {len(gdf)} geometrías. CRS original: {gdf.crs}")

        if gdf.crs is None:
            crs_asignado_por_defecto = "EPSG:6372" # ITRF2008 común para INEGI
            print(f"    ADVERTENCIA: No se detectó CRS. Asignando por defecto: {crs_asignado_por_defecto}. ¡VERIFICAR!")
            gdf.set_crs(crs_asignado_por_defecto, inplace=True)

        crs_destino = "EPSG:4326" # WGS 84
        if gdf.crs != crs_destino:
            print(f"    Reproyectando geometrías de {gdf.crs} a {crs_destino} (WGS 84)...")
            gdf = gdf.to_crs(crs_destino)
            print(f"    Reproyección completada. Nuevo CRS: {gdf.crs}")
        else:
            print(f"    Las geometrías ya están en {crs_destino}. No se reproyecta.")

        print(f"  Guardando GeoDataFrame como GeoParquet en: '{RUTA_GEOPARQUET_SALIDA}'...")
        gdf.to_parquet(RUTA_GEOPARQUET_SALIDA, index=False)
        print(f"    ¡Conversión a GeoParquet completada!")
    except importlib.metadata.PackageNotFoundError as e_arrow:
        print(f"  ¡ERROR DE DEPENDENCIA! Falta 'pyarrow', necesario para 'to_parquet'. Error: {e_arrow}")
        raise
    except Exception as e:
        print(f"  ¡ERROR GENERAL durante la conversión a GeoParquet!: {e}")
        raise

print("\n--- Proceso de Conversión a GeoParquet completado (o intentado). ---")
if os.path.exists(RUTA_GEOPARQUET_SALIDA):
    print(f"Confirmado: GeoParquet listo en: {RUTA_GEOPARQUET_SALIDA}")
else:
    print(f"ADVERTENCIA: GeoParquet NO se encuentra en: {RUTA_GEOPARQUET_SALIDA}.")
    raise FileNotFoundError(f"Archivo GeoParquet no encontrado: {RUTA_GEOPARQUET_SALIDA}")

In [ ]:
# Celda 11: Creación de la Tabla 'manzanas' en DuckDB desde GeoParquet
# -----------------------------------------------------------------
print(f"--- Creando la Tabla 'manzanas' en DuckDB desde el archivo GeoParquet ---")
if not os.path.exists(RUTA_GEOPARQUET_SALIDA):
    print(f"¡ERROR CRÍTICO! No se encontró '{RUTA_GEOPARQUET_SALIDA}'. La tabla 'manzanas' no se puede crear.")
    raise FileNotFoundError(f"GeoParquet no encontrado: {RUTA_GEOPARQUET_SALIDA}")
else:
    print(f"GeoParquet encontrado: '{RUTA_GEOPARQUET_SALIDA}'. Procediendo a crear la tabla 'manzanas'.")
    try:
        con.execute("DROP TABLE IF EXISTS manzanas;")
        print("Tabla 'manzanas' eliminada si existía previamente.")
        sql_create_table_from_geoparquet = f"""
        CREATE TABLE manzanas AS
        SELECT *
        FROM read_parquet('{RUTA_GEOPARQUET_SALIDA}');
        """
        print("\nEjecutando la creación de la tabla 'manzanas' desde GeoParquet...")
        con.execute("BEGIN TRANSACTION;")
        con.execute(sql_create_table_from_geoparquet)
        con.execute("COMMIT;")
        print("¡Tabla 'manzanas' creada y poblada exitosamente!")

        print("\nVerificando la tabla 'manzanas'...")
        num_filas_manzanas = con.execute("SELECT COUNT(*) FROM manzanas;").fetchone()[0]
        print(f"  Número total de filas (manzanas) en 'manzanas': {num_filas_manzanas}")
        if num_filas_manzanas == 0: print("  ADVERTENCIA: La tabla 'manzanas' está vacía.")

        describe_manzanas_df = con.execute("DESCRIBE manzanas;").fetchdf()
        print("\n  Estructura de la tabla 'manzanas':")
        print(describe_manzanas_df)
        geometry_col_info = describe_manzanas_df[describe_manzanas_df['column_name'].str.lower() == 'geometry']
        if not geometry_col_info.empty:
            print(f"\n  Información de 'geometry': {geometry_col_info['column_type'].iloc[0]}")
            if 'GEOMETRY' not in geometry_col_info['column_type'].iloc[0].upper():
                print("    ADVERTENCIA: El tipo de 'geometry' no parece ser GEOMETRY.")
            else:
                print(f"    La columna 'geometry' parece tener un tipo espacial correcto.")
                df_preview_manzanas_geom_only = con.execute("SELECT ST_AsText(geometry) AS geometria_wkt FROM manzanas LIMIT 1;").fetchdf()
                print("    Ejemplo de geometría WKT:", df_preview_manzanas_geom_only['geometria_wkt'].iloc[0][:100] + "..." if not df_preview_manzanas_geom_only.empty else "N/A")

        else:
            print("\n  ADVERTENCIA: No se encontró columna 'geometry'.")
    except Exception as e:
        print(f"¡ERROR GENERAL al crear la tabla 'manzanas'!: {e}")
        try: con.execute("ROLLBACK;")
        except: pass
        raise
print("\n--- Proceso de creación de la tabla 'manzanas' completado (o intentado). ---")

In [ ]:
# Celda 12: Unión de Datos Censales y Geometrías para Crear 'censo_geo'
# --------------------------------------------------------------------
print(f"--- Creando la Tabla 'censo_geo' mediante la unión de 'censo_all' y 'manzanas' ---")
error_preparacion = False
try:
    num_filas_censo_all = con.execute("SELECT COUNT(*) FROM censo_all;").fetchone()[0]
    if num_filas_censo_all == 0: error_preparacion = True; print("ADVERTENCIA: 'censo_all' vacía.")
    else: print(f"'censo_all' verificada, {num_filas_censo_all} filas.")
    num_filas_manzanas = con.execute("SELECT COUNT(*) FROM manzanas;").fetchone()[0]
    if num_filas_manzanas == 0: error_preparacion = True; print("ADVERTENCIA: 'manzanas' vacía.")
    else: print(f"'manzanas' verificada, {num_filas_manzanas} filas.")
except Exception as e:
    error_preparacion = True; print(f"Error verificando tablas: {e}.")

if error_preparacion:
    print("¡ERROR CRÍTICO! Tablas de entrada no listas. 'censo_geo' no se creará.")
    raise ValueError("Tablas de entrada para la unión no están listas.")
else:
    print("\nTablas de entrada ('censo_all' y 'manzanas') listas para la unión.")
    try:
        con.execute("DROP TABLE IF EXISTS censo_geo;")
        print("Tabla 'censo_geo' eliminada si existía previamente.")

        describe_censo_all_df = con.execute("DESCRIBE censo_all;").fetchdf()
        columnas_censo_all = describe_censo_all_df['column_name'].tolist()
        cols_a_excluir_de_censo_select = ['ENTIDAD', 'MUN', 'LOC', 'AGEB', 'MZA'] # Ya están en CVEGEO
        select_cols_censo_str = ", ".join([f"c.\"{col}\"" for col in columnas_censo_all if col not in cols_a_excluir_de_censo_select])

        # Columnas de clave en 'manzanas' (Shapefile del INEGI)
        # Usualmente CVEGEO ya existe, o CVE_ENT, CVE_MUN, etc.
        # Verificamos si CVEGEO existe en 'manzanas', si no, la construimos.
        describe_manzanas_df = con.execute("DESCRIBE manzanas;").fetchdf()
        manzanas_cols = describe_manzanas_df['column_name'].tolist()

        cvegeo_manzana_expr = ""
        if 'CVEGEO' in manzanas_cols:
            cvegeo_manzana_expr = "m.CVEGEO"
            print("  Se usará la columna 'CVEGEO' existente en la tabla 'manzanas'.")
        elif all(col in manzanas_cols for col in ['CVE_ENT', 'CVE_MUN', 'CVE_LOC', 'CVE_AGEB', 'CVE_MZA']):
            print("  Se construirá 'CVEGEO' para la tabla 'manzanas' a partir de sus componentes (CVE_ENT, etc.).")
            cvegeo_manzana_expr = """
                CONCAT(
                    LPAD(CAST(m.CVE_ENT AS VARCHAR), 2, '0'),
                    LPAD(CAST(m.CVE_MUN AS VARCHAR), 3, '0'),
                    LPAD(CAST(m.CVE_LOC AS VARCHAR), 4, '0'),
                    LPAD(CAST(m.CVE_AGEB AS VARCHAR), 4, '0'),
                    LPAD(CAST(m.CVE_MZA AS VARCHAR), 3, '0')
                )
            """
        else:
            print("  ADVERTENCIA: No se encontró 'CVEGEO' ni todas las columnas CVE_* en 'manzanas'. La unión podría fallar o ser incorrecta.")
            print(f"  Columnas disponibles en 'manzanas': {manzanas_cols}")
            raise ValueError("No se pueden generar/encontrar las claves geoestadísticas en la tabla 'manzanas'")


        sql_create_censo_geo = f"""
        CREATE TABLE censo_geo AS
        WITH censo_con_cvegeo AS (
            SELECT
                CONCAT(
                    LPAD(CAST(ENTIDAD AS VARCHAR), 2, '0'),
                    LPAD(CAST(MUN AS VARCHAR), 3, '0'),
                    LPAD(CAST(LOC AS VARCHAR), 4, '0'),
                    LPAD(CAST(AGEB AS VARCHAR), 4, '0'),
                    LPAD(CAST(MZA AS VARCHAR), 3, '0')
                ) AS CVEGEO_CensoCalculado,
                *
            FROM censo_all
        ),
        manzanas_con_cvegeo AS (
            SELECT
                {cvegeo_manzana_expr} AS CVEGEO_ManzanaFuente,
                m.geometry
                -- , m.* EXCLUDE (geometry, CVEGEO) -- si CVEGEO ya existe y no queremos duplicados, o listar explícitamente otras cols de manzana
            FROM manzanas m
        )
        SELECT
            c.CVEGEO_CensoCalculado AS CVEGEO,
            {select_cols_censo_str},
            m_join.geometry
        FROM censo_con_cvegeo c
        INNER JOIN manzanas_con_cvegeo m_join ON c.CVEGEO_CensoCalculado = m_join.CVEGEO_ManzanaFuente;
        """

        print("\nEjecutando la creación de la tabla 'censo_geo' (unión)...")
        con.execute("BEGIN TRANSACTION;")
        con.execute(sql_create_censo_geo)
        con.execute("COMMIT;")
        print("¡Tabla 'censo_geo' creada exitosamente!")

        print("\nVerificando la tabla 'censo_geo'...")
        num_filas_censo_geo = con.execute("SELECT COUNT(*) FROM censo_geo;").fetchone()[0]
        print(f"  Número total de filas en 'censo_geo': {num_filas_censo_geo}")
        if num_filas_censo_geo == 0:
            print("  ADVERTENCIA: 'censo_geo' está vacía. Problemas con la unión o construcción de CVEGEO.")
        else:
            print("\n  Primeras 3 filas de 'censo_geo' (con geometría como WKT abreviado):")
            preview_df_final = con.execute("SELECT CVEGEO, POBTOT, ST_AsText(geometry) AS geometria_wkt FROM censo_geo LIMIT 3;").fetchdf()
            print(preview_df_final)

    except Exception as e:
        print(f"¡ERROR GENERAL al crear la tabla 'censo_geo'!: {e}")
        try: con.execute("ROLLBACK;")
        except: pass
        raise
print("\n--- Proceso de creación de 'censo_geo' completado (o intentado). ---")

In [ ]:
# Ver : https://dbeaver.io/

In [ ]:
# Celda 13: Limpieza Final de Tablas Intermedias y Cierre de la Conexión (Parcial)
# ---------------------------------------------------------------------
# No cerraremos la conexión todavía, ya que la usaremos para la visualización.
# Pero sí podemos limpiar tablas intermedias.

print(f"--- Iniciando Limpieza Final de Tablas Intermedias ---")
ELIMINAR_TABLAS_INTERMEDIAS = True
if ELIMINAR_TABLAS_INTERMEDIAS:
    print("\nEliminando tablas intermedias ('censo_all' y 'manzanas')...")
    try:
        con.execute("DROP TABLE IF EXISTS censo_all;")
        print("  Tabla 'censo_all' eliminada (si existía).")
        con.execute("DROP TABLE IF EXISTS manzanas;")
        print("  Tabla 'manzanas' eliminada (si existía).")
        print("Limpieza de tablas intermedias completada.")
    except Exception as e:
        print(f"  Ocurrió un error durante la eliminación de tablas intermedias: {e}")
else:
    print("\nSe conservarán las tablas intermedias 'censo_all' y 'manzanas'.")

print("\nVerificando tablas finales en la base de datos...")
try:
    tablas_finales_df = con.execute("SHOW TABLES;").fetchdf()
    if not tablas_finales_df.empty:
        print("Tablas actualmente en la base de datos:")
        print(tablas_finales_df)
        if 'censo_geo' in tablas_finales_df['name'].tolist():
            print("\n  ¡La tabla principal 'censo_geo' está presente!")
        else:
            print("\n  ADVERTENCIA: ¡'censo_geo' NO se encontró!")
    else:
        print("No se encontraron tablas.")
except Exception as e:
    print(f"  Error al mostrar las tablas: {e}")

print("\n--- Limpieza Parcial Completada. La conexión a DuckDB se mantiene abierta para la visualización. ---")

# La Celda 14 original (exportar a gpkg) se puede ejecutar si se desea una salida de archivo.
# Por ahora la omitimos para ir directo a la visualización desde DuckDB.
# print("\n--- Si se desea exportar 'censo_geo' a GeoPackage, se puede añadir el código de la Celda 14 aquí ---")

## Visualización Interactiva de Datos Censales de Aguascalientes con Folium

Ahora que hemos procesado y unido los datos censales y geoespaciales para el estado de Aguascalientes en la tabla `censo_geo` dentro de nuestra base de datos DuckDB, podemos proceder a visualizarlos. Utilizaremos `folium`, una potente librería de Python que nos permite crear mapas interactivos basados en Leaflet.js.

**Pasos para la Visualización:**

1.  **Lectura de Datos desde DuckDB:**
    *   Nos conectaremos a la base de datos DuckDB (si la conexión se hubiera cerrado, la reabriríamos).
    *   Ejecutaremos una consulta SQL para extraer los datos de la tabla `censo_geo`. Es importante obtener la columna de geometría en un formato que GeoPandas pueda interpretar fácilmente, como WKB (Well-Known Binary) o WKT (Well-Known Text). DuckDB, con su extensión espacial, puede convertir las geometrías a estos formatos.
    *   Los datos leídos se cargarán en un GeoDataFrame de GeoPandas.

2.  **Preparación del GeoDataFrame:**
    *   Aseguraremos que la columna de geometría esté correctamente interpretada por GeoPandas.
    *   Verificaremos que el Sistema de Coordenadas de Referencia (CRS) sea `EPSG:4326` (WGS 84), que es el estándar para la mayoría de las herramientas de mapeo web, incluido Folium. Nuestros datos ya deberían estar en este CRS debido al paso de conversión a GeoParquet.
    *   Seleccionaremos la variable que deseamos visualizar (por ejemplo, `POBTOT` para la población total).

3.  **Creación del Mapa Base con Folium:**
    *   Calcularemos un punto central aproximado para el estado de Aguascalientes para centrar nuestro mapa. Esto se puede hacer tomando el centroide de la unión de todas las geometrías o el centroide de una geometría representativa.
    *   Inicializaremos un objeto `folium.Map` con esta ubicación central y un nivel de zoom adecuado.

4.  **Añadir Capa Coroplética (Choropleth):**
    *   Utilizaremos la función `folium.Choropleth` para dibujar las manzanas y colorearlas según el valor de la variable seleccionada.
    *   Esta función requiere:
        *   `geo_data`: Los datos geoespaciales en formato GeoJSON (GeoPandas puede convertir a este formato).
        *   `data`: El DataFrame que contiene los valores a visualizar.
        *   `columns`: Las columnas que especifican el identificador de la geometría y el valor a mapear.
        *   `key_on`: Una ruta en la estructura GeoJSON para encontrar el identificador que coincida con el del DataFrame.
        *   `fill_color`: Una paleta de colores (por ejemplo, 'YlGnBu', 'RdYlGn', etc.).
        *   `fill_opacity`, `line_opacity`: Para la transparencia.
        *   `legend_name`: El título para la leyenda del mapa.

5.  **Mostrar el Mapa Interactivo:**
    *   Al final, mostraremos el objeto mapa de Folium. En un entorno como Google Colab o un Jupyter Notebook, esto renderizará un mapa interactivo directamente en la salida de la celda.

Este enfoque nos permitirá explorar visualmente la distribución de características demográficas o de vivienda a lo largo de las manzanas de Aguascalientes.


In [ ]:
# Celda 15: Visualización Interactiva con Folium (Usando WKT para Geometrías)
# --------------------------------------------------------------------------

import duckdb
import geopandas as gpd
import pandas as pd
from shapely import wkt # Cambiado de wkb a wkt
from shapely import errors as shapely_errors # Para manejo de errores de shapely
import folium
import os
# display para asegurar el renderizado en Colab
from IPython.display import display


print("--- Iniciando Visualización Interactiva con Folium para Aguascalientes (Usando WKT) ---")

# --- Variables de Configuración (Requeridas si esta celda se ejecuta de forma aislada) ---
CODIGO_ESTADO_STR = "01"
NOMBRE_ESTADO = "Aguascalientes"
DIR_BASE_INEGI = "./inegi_data_ags"
DB_FILENAME = f"inegi_analisis_{CODIGO_ESTADO_STR}.duckdb"
DB_FILE_PATH = os.path.join(DIR_BASE_INEGI, DB_FILENAME)

try:
    # 1. Establecer/Reestablecer la Conexión a DuckDB
    # ---------------------------------------------
    print(f"Asegurando conexión a DuckDB en: {DB_FILE_PATH}")
    if not os.path.exists(DB_FILE_PATH):
        print(f"  ¡ERROR CRÍTICO! El archivo de base de datos no existe en: {DB_FILE_PATH}")
        raise FileNotFoundError(f"Base de datos no encontrada: {DB_FILE_PATH}")

    # Verifica si la conexión 'con' ya existe y es una instancia de duckdb.DuckDBPyConnection
    if con is not None and isinstance(con, duckdb.DuckDBPyConnection):
        print("Ya existe una conexión a DuckDB.")
    else:
        print("No existe una conexión a DuckDB o no es válida. Conectando...")
        con = duckdb.connect(database=DB_FILE_PATH, read_only=True)
        print("Conexión establecida (solo lectura).")

    con.execute("LOAD spatial;")
    print("  Extensión 'spatial' cargada.")

    # 2. Leer datos de 'censo_geo' desde DuckDB (usando ST_AsText para WKT)
    # --------------------------------------------------------------------
    print("Leyendo datos de la tabla 'censo_geo' desde DuckDB (geometría como WKT)...")

    query = """
    SELECT
        CVEGEO,
        NOM_MUN,
        NOM_LOC,
        POBTOT,
        VIVTOT,
        TVIVHAB,
        VPH_INTER,
        VPH_PC,
        VPH_CEL,
        VPH_STVP,
        VPH_SPMVPI,
        VPH_CVJ,
        GRAPROES,
        ST_AsText(geometry) AS geom_wkt -- Cambiado a ST_AsText
    FROM censo_geo
    WHERE POBTOT IS NOT NULL AND POBTOT > 0;
    """
    df_from_duckdb = con.execute(query).fetch_df() # Cambié el nombre para claridad

    if df_from_duckdb.empty:
        print("  ADVERTENCIA: No se encontraron datos en 'censo_geo' para visualizar.")
    else:
        print(f"  Datos leídos: {len(df_from_duckdb)} manzanas con población > 0.")

        # Convertir la columna WKT a geometrías de GeoPandas
        # Adaptado del ejemplo que funcionó en el script anterior
        def parse_wkt_geometry(wkt_string):
            if wkt_string is None or not isinstance(wkt_string, str) or wkt_string.strip() == "":
                return None
            try:
                return wkt.loads(wkt_string)
            except shapely_errors.GEOSException as e_wkt:
                # print(f"    Advertencia: Error al leer WKT '{wkt_string[:50]}...': {e_wkt}. Se devolverá None.")
                return None # Silenciar warnings por ahora para no inundar la salida
            except Exception as e_gen:
                # print(f"    Advertencia: Error inesperado al procesar WKT: {e_gen}. Se devolverá None.")
                return None

        # Aplicar la función de parseo
        parsed_geometries = df_from_duckdb['geom_wkt'].apply(parse_wkt_geometry)

        # Crear el GeoDataFrame
        gdf_ags = gpd.GeoDataFrame(df_from_duckdb.drop(columns=['geom_wkt']),
                                   geometry=parsed_geometries,
                                   crs="EPSG:4326") # El CRS ya es 4326 por la transformación en Celda 10

        print(f"  GeoDataFrame creado. CRS: {gdf_ags.crs}")

        # Contar geometrías nulas después del parseo
        null_geom_count = gdf_ags.geometry.isnull().sum()
        if null_geom_count > 0:
            print(f"  ADVERTENCIA: {null_geom_count} geometrías son nulas después del parseo de WKT.")
            gdf_ags.dropna(subset=['geometry'], inplace=True) # Eliminar filas con geometrías nulas
            print(f"  Filas con geometrías nulas eliminadas. Quedan {len(gdf_ags)} filas.")


        # 3. Preparación del GeoDataFrame
        # -------------------------------
        variable_a_visualizar = 'POBTOT'

        if variable_a_visualizar not in gdf_ags.columns:
            print(f"  ¡ERROR! La columna '{variable_a_visualizar}' no existe.")
            variable_a_visualizar = None
        else:
            gdf_ags[variable_a_visualizar] = pd.to_numeric(gdf_ags[variable_a_visualizar], errors='coerce')
            gdf_ags.dropna(subset=[variable_a_visualizar], inplace=True) # Eliminar filas si la variable es NaN

        if gdf_ags.empty or variable_a_visualizar is None:
            print("  No hay datos válidos para visualizar después de la limpieza.")
        else:
            # 4. Creación del Mapa Base con Folium
            # -----------------------------------
            bounds = gdf_ags.total_bounds
            map_center_lat = (bounds[1] + bounds[3]) / 2
            map_center_lon = (bounds[0] + bounds[2]) / 2

            print(f"  Centro del mapa: Lat={map_center_lat:.4f}, Lon={map_center_lon:.4f}")

            m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=12, tiles="CartoDB positron")

            # 5. Añadir Capa Coroplética
            # --------------------------
            if 'CVEGEO' in gdf_ags.columns and gdf_ags['CVEGEO'].is_unique:
                 key_column = 'CVEGEO'
            else:
                 gdf_ags['folium_idx'] = range(len(gdf_ags))
                 key_column = 'folium_idx'

            try:
                bins = list(gdf_ags[variable_a_visualizar].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
                bins = sorted(list(set(bins))) # Asegurar unicidad y orden
                if len(bins) < 2: bins = 6 # Si no hay suficiente variación, usar un número fijo
            except Exception: # Si el cálculo de cuantiles falla por alguna razón
                bins = 6 # Default a un número fijo de bins

            folium.Choropleth(
                geo_data=gdf_ags.to_json(),
                name=f'Coropleta de {variable_a_visualizar}',
                data=gdf_ags,
                columns=[key_column, variable_a_visualizar],
                key_on=f'feature.properties.{key_column}',
                fill_color='YlOrRd',
                fill_opacity=0.7,
                line_opacity=0.2, # Líneas de borde más sutiles
                legend_name=f'{variable_a_visualizar} por Manzana en {NOMBRE_ESTADO}',
                highlight=True,
                bins=bins
            ).add_to(m)

            # Tooltips
            tooltip_fields = ['CVEGEO', 'NOM_MUN', 'NOM_LOC', variable_a_visualizar, 'VIVTOT', 'VPH_INTER', 'VPH_PC', 'VPH_CEL', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'GRAPROES']
            tooltip_aliases = [
                'Clave:', 'Municipio:', 'Localidad:',
                f'{variable_a_visualizar.replace("_", " ")}:',
                'Viv. Totales:', 'Viv. Internet:', 'Viv. PC:', 'Viv. Celular:',
                'Viv. TV Paga:', 'Viv. Streaming:', 'Viv. Consola:',
                'Escolaridad Prom.:'
            ]

            tooltip_fields_existentes = [field for field in tooltip_fields if field in gdf_ags.columns]
            # Crear aliases solo para los campos existentes para evitar errores de índice
            tooltip_aliases_existentes = [tooltip_aliases[tooltip_fields.index(field)] for field in tooltip_fields_existentes]


            if tooltip_fields_existentes:
                 # Para Folium, es común que el tooltip se añada a una capa GeoJson separada
                 # que puede ser transparente si solo se quiere el tooltip.
                 folium.features.GeoJson(
                    data=gdf_ags.to_json(), # GeoJSON de tus datos
                    style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent', 'weight':0}, # Hace la capa invisible
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=tooltip_fields_existentes,
                        aliases=tooltip_aliases_existentes,
                        sticky=False, # El tooltip sigue al cursor
                        localize=True, # Formatea números y fechas según la configuración local
                        style=("background-color: white; color: black; font-family: arial; font-size: 12px; padding: 10px;")
                    ),
                    name="Información detallada (hover)"
                ).add_to(m)


            folium.LayerControl().add_to(m)

            # 6. Mostrar el Mapa Interactivo
            # ------------------------------
            print("\nGenerando mapa interactivo de Folium...")
            display(m)

except duckdb.Error as e_duck:
    print(f"  Error de DuckDB: {e_duck}")
except FileNotFoundError as e_file:
    print(f"  Error de Archivo: {e_file}")
except Exception as e_general:
    print(f"  Ocurrió un error general durante la visualización: {e_general}")
    import traceback
    traceback.print_exc()
finally:
    if con:
        try:
            con.close()
            print("\nConexión a DuckDB cerrada al finalizar la visualización.")
        except Exception as e_close:
            print(f"  Error al intentar cerrar la conexión a DuckDB: {e_close}")

print("\n--- Fin del Script de Visualización ---")

**Explicación Detallada de las Llamadas a `folium` en el Código:**

`folium` es una librería de Python que facilita la visualización de datos geoespaciales de manera interactiva en un mapa Leaflet.js. Esta herramienta toma datos geográficos y los atributos que se desean mostrar, y genera un archivo HTML que contiene un mapa interactivo explorable con zoom, paneo, y a menudo, con información emergente (tooltips).

En el script, `folium` se utiliza principalmente en dos celdas:

1.  **Celda 15: Visualización de Manzanas Individuales (mapa `m`)**
2.  **Celda 16: Visualización de Agregación Hexagonal (mapa `m_hex`)**

Ambas celdas siguen una estructura similar para la creación del mapa:

1.  **Creación del Mapa Base (`folium.Map`)**
2.  **Adición de Capas de Datos (principalmente `folium.Choropleth`)**
3.  **Adición de Información Emergente (Tooltips con `folium.features.GeoJson` y `folium.features.GeoJsonTooltip`)**
4.  **Adición de Control de Capas (`folium.LayerControl`)**
5.  **Visualización del Mapa (`display(m)`)**

A continuación, se analizan los componentes clave:

---

**1. `folium.Map()`**

*   **Qué esperar:** Esta es la primera llamada a `folium` y crea el lienzo base del mapa. Se visualizará un mapa del mundo (o una región específica si ya está centrado) con los controles básicos de zoom y paneo.
*   **Funcionamiento de los parámetros (ejemplo de Celda 15):**
    ```python
    m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=12, tiles="CartoDB positron")
    ```
    *   `location=[map_center_lat, map_center_lon]`:
        *   **Qué es:** Una lista o tupla con dos números: `[latitud, longitud]`. Define el punto central donde se enfocará el mapa inicialmente.
        *   **En el código:** `map_center_lat` y `map_center_lon` se calculan a partir de los límites (`total_bounds`) del GeoDataFrame (`gdf_ags` o `gdf_hex_vis`), asegurando que el mapa se centre en los datos de Aguascalientes.
    *   `zoom_start=12`:
        *   **Qué es:** Un número entero que define el nivel de zoom inicial del mapa. Números más altos significan un mayor acercamiento.
        *   **En el código:** `12` es un nivel de zoom razonable para observar detalles a nivel de ciudad o región.
    *   `tiles="CartoDB positron"`:
        *   **Qué es:** Define el estilo del mapa base (el fondo cartográfico).
        *   **En el código:** `"CartoDB positron"` es un estilo de mapa claro y minimalista. Otros estilos populares incluyen `"OpenStreetMap"`, `"Stamen Terrain"`, `"Stamen Toner"`, etc.
        *   **Qué esperar:** Un mapa con calles, cuerpos de agua, etc., con el estilo "CartoDB positron".

---

**2. `folium.Choropleth()`**

*   **Qué esperar:** Esta es la función clave para crear mapas coropléticos. Dibuja las geometrías de los datos (manzanas en Celda 15, hexágonos en Celda 16) y las colorea según los valores de una variable específica (ej. `POBTOT`). También añade una leyenda para interpretar los colores.
*   **Funcionamiento de los parámetros (ejemplo de Celda 15):**
    ```python
    folium.Choropleth(
        geo_data=gdf_ags.to_json(),
        name=f'Coropleta de {variable_a_visualizar}',
        data=gdf_ags,
        columns=[key_column, variable_a_visualizar],
        key_on=f'feature.properties.{key_column}',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f'{variable_a_visualizar} por Manzana en {NOMBRE_ESTADO}',
        highlight=True,
        bins=bins
    ).add_to(m)
    ```
    *   `geo_data=gdf_ags.to_json()`:
        *   **Qué es:** Los datos geoespaciales que contienen las geometrías a dibujar. `folium` espera esto en formato GeoJSON.
        *   **En el código:** `gdf_ags` (o `gdf_hex_vis`) es el GeoDataFrame. El método `.to_json()` lo convierte al formato GeoJSON necesario.
        *   **Qué esperar:** Las manzanas (o hexágonos) de Aguascalientes dibujadas en el mapa.
    *   `name=f'Coropleta de {variable_a_visualizar}'` (y similar en Celda 16):
        *   **Qué es:** Un nombre para esta capa. Es útil si existen múltiples capas, ya que aparecerá en el `LayerControl`.
        *   **Qué esperar:** Este texto identificará la capa en el control de capas.
    *   `data=gdf_ags`:
        *   **Qué es:** El DataFrame (o GeoDataFrame) que contiene los datos (la variable) que se usarán para colorear las geometrías.
        *   **En el código:** Es el mismo `gdf_ags` que también proporciona las geometrías, pero podría ser un DataFrame de Pandas diferente si las geometrías y los datos estuvieran separados.
    *   `columns=[key_column, variable_a_visualizar]`:
        *   **Qué es:** Una lista de dos cadenas.
            *   La primera (`key_column`): El nombre de la columna en `data` (y en las propiedades de `geo_data`) que sirve como identificador único para unir las geometrías con los datos. En el código, es `CVEGEO` o un `folium_idx` si `CVEGEO` no es único o no existe.
            *   La segunda (`variable_a_visualizar`): El nombre de la columna en `data` cuyos valores determinarán el color de cada geometría (ej. `POBTOT`).
        *   **Qué esperar:** `folium` usará `key_column` para encontrar la geometría correcta y luego usará el valor de `variable_a_visualizar` para esa geometría para asignarle un color.
    *   `key_on=f'feature.properties.{key_column}'`:
        *   **Qué es:** Una cadena que le indica a `folium` dónde encontrar el identificador (la `key_column`) dentro de la estructura del `geo_data` (GeoJSON). La estructura típica es `feature.properties.NOMBRE_DE_LA_PROPIEDAD`.
    *   `fill_color='YlOrRd'` (o `'BuPu'` en Celda 16):
        *   **Qué es:** El esquema de color a utilizar. Estos son nombres de paletas de ColorBrewer.
        *   **Qué esperar:** Las geometrías se colorearán usando una gradación de Amarillo-Naranja-Rojo (o Azul-Púrpura).
    *   `fill_opacity=0.7` (o `0.75`):
        *   **Qué es:** La opacidad del relleno de color (0=transparente, 1=opaco).
    *   `line_opacity=0.2` (o `0.3`):
        *   **Qué es:** La opacidad de las líneas de borde de las geometrías.
    *   `legend_name=f'{variable_a_visualizar} por Manzana en {NOMBRE_ESTADO}'`:
        *   **Qué es:** El título que aparecerá sobre la leyenda del mapa.
        *   **Qué esperar:** Una leyenda en el mapa que ayuda a entender qué rango de valores representa cada color.
    *   `highlight=True`:
        *   **Qué es:** Si es `True`, las geometrías se resaltarán al pasar el cursor sobre ellas.
    *   `bins=bins`:
        *   **Qué es:** Define cómo se agrupan los datos para la asignación de colores. Puede ser un número entero (para N clases de igual tamaño) o una lista de valores de corte (para clases personalizadas).
        *   **En el código:** `bins` se calcula usando cuantiles (`gdf_ags[variable_a_visualizar].quantile(...)`) para intentar una distribución de colores más equitativa.
    *   `show=show_this_layer_by_default` (solo en Celda 16):
        *   **Qué es:** Un booleano que indica si la capa debe ser visible por defecto cuando el mapa se carga. Útil cuando hay múltiples capas coropléticas.
    *   `.add_to(m)`:
        *   **Qué es:** Un método que añade esta capa coroplética al objeto mapa `m` creado anteriormente.

---

**3. `folium.features.GeoJson()` y `folium.features.GeoJsonTooltip()` (para Tooltips)**

*   **Qué esperar:** Estas funciones trabajan juntas para mostrar información adicional cuando se pasa el cursor sobre una geometría (manzana o hexágono). Aparecerá una pequeña ventana emergente (tooltip) con los datos que se especifiquen.
*   **Funcionamiento de los parámetros (ejemplo de Celda 15):**
    ```python
    folium.features.GeoJson(
        data=gdf_ags.to_json(),
        style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent', 'weight':0},
        tooltip=folium.features.GeoJsonTooltip(
            fields=tooltip_fields_existentes,
            aliases=tooltip_aliases_existentes,
            sticky=False,
            localize=True,
            style=("background-color: white; color: black; font-family: arial; font-size: 12px; padding: 10px;")
        ),
        name="Información detallada (hover)"
    ).add_to(m)
    ```
    *   **`folium.features.GeoJson()`:**
        *   `data=gdf_ags.to_json()`: Nuevamente, las geometrías en formato GeoJSON.
        *   `style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent', 'weight':0}`:
            *   **Qué es:** Una función que define el estilo de esta capa GeoJSON.
            *   **En el código:** Se usa para hacer esta capa completamente invisible, ya que su único propósito es servir de "ancla" para los tooltips. Los colores y las formas ya están definidos por la capa `Choropleth`.
        *   `tooltip=folium.features.GeoJsonTooltip(...)`:
            *   **Qué es:** Aquí se anida el objeto que define el contenido y comportamiento del tooltip.
        *   `name="Información detallada (hover)"`: Nombre para esta capa en el control de capas.
    *   **`folium.features.GeoJsonTooltip()`:**
        *   `fields=tooltip_fields_existentes`:
            *   **Qué es:** Una lista de los nombres de las columnas del `gdf_ags` (o `gdf_hex_vis`) que se quieren mostrar en el tooltip.
            *   **En el código:** `tooltip_fields_existentes` se asegura de que solo se incluyan columnas que realmente existen en los datos.
        *   `aliases=tooltip_aliases_existentes`:
            *   **Qué es:** Una lista de etiquetas más amigables que se mostrarán en lugar de los nombres de columna crudos. Deben estar en el mismo orden que `fields`.
        *   `sticky=False`:
            *   **Qué es:** Si es `False`, el tooltip sigue el cursor del ratón mientras está sobre la geometría. Si es `True`, se fija a un punto de la geometría.
        *   `localize=True`:
            *   **Qué es:** Intenta formatear números y fechas según la configuración regional del navegador.
        *   `style=`:
            *   **Qué es:** Una cadena CSS para darle estilo al tooltip (color de fondo, fuente, etc.).

---

**4. `folium.LayerControl()`**

*   **Qué esperar:** Añade un control (generalmente en la esquina superior derecha del mapa) que permite activar o desactivar la visibilidad de las diferentes capas que se han añadido al mapa (las capas `Choropleth` y `GeoJson` que tienen un parámetro `name`).
*   **Funcionamiento de los parámetros (ejemplo de Celda 16):**
    ```python
    folium.LayerControl(collapsed=False).add_to(m_hex)
    ```
    *   `collapsed=False`:
        *   **Qué es:** Si es `False`, el control de capas se muestra expandido por defecto. Si es `True`, está colapsado y se necesita hacer clic para ver las capas.
    *   `.add_to(m_hex)`: Añade el control al mapa `m_hex`.

---

**5. `display(m)`**

*   **Qué esperar:** En entornos como Jupyter Notebook o Google Colab, esta función es la que efectivamente renderiza el objeto mapa `m` (o `m_hex`) como una salida interactiva en la celda.
*   **Funcionamiento:** `display` es una función de IPython que puede mostrar diferentes tipos de objetos de forma enriquecida. Para los mapas de `folium`, los muestra como el mapa HTML interactivo.

---

**En Resumen del Proceso con `folium`:**

1.  **Preparación de datos:** Se leen los datos geoespaciales y los atributos a visualizar en un GeoDataFrame de GeoPandas (ej. `gdf_ags`, `gdf_hex_vis`). Se asegura que el Sistema de Referencia de Coordenadas (CRS) sea `EPSG:4326`.
2.  **Creación del mapa base:** `folium.Map()` proporciona el lienzo inicial.
3.  **Adición de datos coloreados:** `folium.Choropleth()` dibuja las geometrías (manzanas/hexágonos) y las colorea según una variable, añadiendo una leyenda para la interpretación.
4.  **Adición de interactividad:** `folium.features.GeoJson()` junto con `folium.features.GeoJsonTooltip()` permite que aparezca información útil cuando se pasa el cursor sobre las geometrías.
5.  **Inclusión de control de capas:** `folium.LayerControl()` ofrece la opción de mostrar u ocultar capas si se han definido varias.
6.  **Visualización del resultado:** `display(m)` renderiza el mapa interactivo final.

El resultado final es un mapa HTML incrustado en el notebook, donde se puede hacer zoom, desplazarse, observar cómo se distribuye la variable de interés (población, viviendas con internet, etc.) a través de las diferentes áreas geográficas de Aguascalientes, y obtener detalles específicos al pasar el cursor por encima de los elementos del mapa.

In [ ]:
pip install geohexgrid --quiet

In [ ]:
# Celda 16: Visualización Exclusiva de Agregación Hexagonal (Nuevo Intento para AssertionError)
# ---------------------------------------------------------------------------------------------

import duckdb
import geopandas as gpd
import pandas as pd
from shapely import wkt, wkb
from shapely.geometry import Point
import folium
import os
from IPython.display import display
import geohexgrid as ghg
import numpy as np

print("--- Iniciando Visualización Exclusiva de Agregación Hexagonal (Nuevo Intento AssertionError) ---")

# --- Variables de Configuración ---
CODIGO_ESTADO_STR = "01"
NOMBRE_ESTADO = "Aguascalientes"
DIR_BASE_INEGI = "./inegi_data_ags"
DB_FILENAME = f"inegi_analisis_{CODIGO_ESTADO_STR}.duckdb"
DB_FILE_PATH = os.path.join(DIR_BASE_INEGI, DB_FILENAME)
RADIO_HEXAGONO_METROS = 350
CRS_PROYECTADO_MEXICO = "EPSG:6372"
CRS_GEOGRAFICO = "EPSG:4326"

con = None

try:
    # PARTE A: CREACIÓN/VERIFICACIÓN DE DATOS HEXAGONALES EN DUCKDB (Se mantiene igual)
    # ================================================================================
    print(f"Conectando a DuckDB (RW) en: {DB_FILE_PATH} para preparar datos hexagonales...")
    if not os.path.exists(DB_FILE_PATH):
        raise FileNotFoundError(f"Base de datos no encontrada: {DB_FILE_PATH}.")

    con = duckdb.connect(database=DB_FILE_PATH, read_only=False)
    print("  Conexión a DuckDB establecida.")
    con.execute("LOAD spatial;")
    print("  Extensión 'spatial' cargada.")

    table_check_censo_geo = con.execute("SELECT table_name FROM information_schema.tables WHERE table_name = 'censo_geo';").fetch_df()
    if table_check_censo_geo.empty:
        raise ValueError("La tabla base 'censo_geo' no existe.")

    print("Preparando tabla de centroides de manzanas ('manzana_centroids_ags')...")
    con.execute("""
    CREATE TABLE IF NOT EXISTS manzana_centroids_ags AS
    SELECT
        CVEGEO, POBTOT, VIVTOT, TVIVHAB, VPH_INTER, VPH_PC,
        VPH_CEL, VPH_STVP, VPH_SPMVPI, VPH_CVJ, GRAPROES,
        ST_Centroid(geometry) as centroid_geom
    FROM censo_geo
    WHERE POBTOT IS NOT NULL AND POBTOT > 0 AND geometry IS NOT NULL AND ST_IsValid(geometry);
    """)
    num_centroids = con.execute("SELECT COUNT(*) FROM manzana_centroids_ags").fetchone()[0]
    if num_centroids == 0:
        con.execute("DROP TABLE IF EXISTS manzana_centroids_ags;")
        con.execute("""
        CREATE TABLE manzana_centroids_ags AS SELECT
            CVEGEO, POBTOT, VIVTOT, TVIVHAB, VPH_INTER, VPH_PC, VPH_CEL, VPH_STVP, VPH_SPMVPI, VPH_CVJ, GRAPROES,
            ST_Centroid(geometry) as centroid_geom
        FROM censo_geo WHERE POBTOT IS NOT NULL AND POBTOT > 0 AND geometry IS NOT NULL AND ST_IsValid(geometry);""")
        num_centroids = con.execute("SELECT COUNT(*) FROM manzana_centroids_ags").fetchone()[0]
        if num_centroids == 0: raise ValueError("No se generaron centroides válidos desde 'censo_geo'.")
    print(f"  Tabla 'manzana_centroids_ags' lista/creada con {num_centroids} centroides.")

    print("Preparando rejilla hexagonal ('hex_grid_ags')...")
    table_check_hex_grid = con.execute("SELECT table_name FROM information_schema.tables WHERE table_name = 'hex_grid_ags';").fetch_df()
    if table_check_hex_grid.empty:
        print("  Tabla 'hex_grid_ags' no encontrada, generándola...")
        df_centroids_for_grid = con.execute("SELECT ST_AsText(centroid_geom) AS geom_wkt FROM manzana_centroids_ags WHERE centroid_geom IS NOT NULL;").fetch_df()
        if df_centroids_for_grid.empty:
            raise ValueError("No se pudieron extraer centroides WKT para generar la rejilla.")

        def parse_wkt_for_grid(wkt_string):
            if wkt_string is None or not isinstance(wkt_string, str) or wkt_string.strip() == "": return None
            try: return wkt.loads(wkt_string)
            except: return None

        df_centroids_for_grid['geometry'] = df_centroids_for_grid['geom_wkt'].apply(parse_wkt_for_grid)
        gdf_centroids_for_grid = gpd.GeoDataFrame(df_centroids_for_grid.drop(columns=['geom_wkt']), geometry='geometry', crs=CRS_GEOGRAFICO)
        gdf_centroids_for_grid.dropna(subset=['geometry'], inplace=True)

        if gdf_centroids_for_grid.empty:
            raise ValueError("GeoDataFrame de centroides (desde WKT) para la rejilla está vacío.")

        grid_hex_proj = ghg.make_grid_from_gdf(gdf_centroids_for_grid.to_crs(CRS_PROYECTADO_MEXICO), R=RADIO_HEXAGONO_METROS)
        grid_hex = grid_hex_proj.to_crs(CRS_GEOGRAFICO)
        print(f"  Rejilla hexagonal generada con {len(grid_hex)} celdas.")

        grid_hex['geom_wkt_hex'] = grid_hex['geometry'].apply(lambda g: g.wkt if g else None)
        grid_hex_for_duckdb = grid_hex[['cell_id', 'geom_wkt_hex']].copy()
        grid_hex_for_duckdb.dropna(subset=['geom_wkt_hex'], inplace=True)

        con.execute("CREATE TABLE hex_grid_ags (cell_id VARCHAR, geometry GEOMETRY);")
        if not grid_hex_for_duckdb.empty:
            for index, row in grid_hex_for_duckdb.iterrows():
                con.execute("INSERT INTO hex_grid_ags VALUES (?, ST_GeomFromText(?))", [row['cell_id'], row['geom_wkt_hex']])
            print("  Rejilla hexagonal cargada a 'hex_grid_ags' usando WKT.")
        else:
            raise ValueError("Rejilla hexagonal generada estaba vacía después de la conversión a WKT.")
    else:
        print("  Tabla 'hex_grid_ags' ya existe.")

    print("Preparando tabla de agregación hexagonal ('hex_final_ags')...")
    variables_numericas_duck = ['POBTOT', 'VIVTOT', 'TVIVHAB', 'VPH_INTER', 'VPH_PC', 'VPH_CEL', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'GRAPROES']
    sum_expressions = [f"SUM(COALESCE(c.{var}, 0)) AS {var}_hex" for var in variables_numericas_duck]

    query_hex_final = f"""
    CREATE OR REPLACE TABLE hex_final_ags AS
    SELECT
        h.cell_id,
        h.geometry AS hex_geom,
        COUNT(c.CVEGEO) AS num_manzanas_en_hex,
        {', '.join(sum_expressions)}
    FROM hex_grid_ags h
    LEFT JOIN manzana_centroids_ags c
        ON ST_Intersects(c.centroid_geom, h.geometry)
    GROUP BY h.cell_id, h.geometry;
    """
    con.execute(query_hex_final)
    num_hex_final = con.execute("SELECT COUNT(*) FROM hex_final_ags").fetchone()[0]
    if num_hex_final == 0:
        raise ValueError("La tabla 'hex_final_ags' se generó vacía.")
    print(f"  Tabla 'hex_final_ags' lista/creada con {num_hex_final} hexágonos agregados.")

    # PARTE B: VISUALIZACIÓN CON FOLIUM
    # =================================
    print("\nLeyendo datos de hexágonos agregados ('hex_final_ags') para visualización...")

    select_hex_vars_display = [f"{var}_hex" for var in variables_numericas_duck]
    query_hex_vis_display = f"""
    SELECT
        cell_id,
        num_manzanas_en_hex,
        {', '.join(select_hex_vars_display)},
        ST_AsText(hex_geom) AS geom_wkt
    FROM hex_final_ags
    WHERE num_manzanas_en_hex > 0;
    """

    df_hex_vis_duck = con.execute(query_hex_vis_display).fetch_df()

    if df_hex_vis_duck.empty:
        raise ValueError("No se extrajeron datos de hexágonos para visualización.")

    def parse_wkt_for_viz(wkt_string):
        if wkt_string is None or not isinstance(wkt_string, str) or wkt_string.strip() == "": return None
        try: return wkt.loads(wkt_string)
        except: return None

    df_hex_vis_duck['geometry'] = df_hex_vis_duck['geom_wkt'].apply(parse_wkt_for_viz)
    gdf_hex_vis = gpd.GeoDataFrame(df_hex_vis_duck.drop(columns=['geom_wkt']), geometry='geometry', crs=CRS_GEOGRAFICO)
    gdf_hex_vis.dropna(subset=['geometry'], inplace=True)

    for var_hex in select_hex_vars_display + ['num_manzanas_en_hex']:
        if var_hex in gdf_hex_vis.columns:
            gdf_hex_vis[var_hex] = pd.to_numeric(gdf_hex_vis[var_hex], errors='coerce')
            gdf_hex_vis[var_hex] = gdf_hex_vis[var_hex].replace([np.inf, -np.inf], np.nan).fillna(0)

    if gdf_hex_vis.empty:
        raise ValueError("GeoDataFrame de Hexágonos para visualización está vacío después de la limpieza.")

    print(f"  GeoDataFrame de Hexágonos para visualización: {len(gdf_hex_vis)} filas. CRS: {gdf_hex_vis.crs}")

    bounds_map_hex = gdf_hex_vis.total_bounds
    map_center_lat_hex = (bounds_map_hex[1] + bounds_map_hex[3]) / 2
    map_center_lon_hex = (bounds_map_hex[0] + bounds_map_hex[2]) / 2
    m_hex = folium.Map(location=[map_center_lat_hex, map_center_lon_hex], zoom_start=12, tiles="CartoDB positron")
    print(f"  Mapa base para hexágonos creado, centrado en: Lat={map_center_lat_hex:.4f}, Lon={map_center_lon_hex:.4f}")

    variables_a_mapear_hex_dict = {
        'POBTOT_hex': 'Población Total (Hex)', 'VPH_INTER_hex': 'Viv. Internet (Hex)',
        'GRAPROES_hex': 'Escolaridad Prom. (Hex)', 'num_manzanas_en_hex': 'Num. Manzanas (Hex)'
    }
    default_hex_variable_to_show = 'POBTOT_hex'

    # ***** INICIO DEL CAMBIO PRINCIPAL PARA EVITAR ASSERTIONERROR *****
    for var_col_name, layer_display_name in variables_a_mapear_hex_dict.items():
        if var_col_name not in gdf_hex_vis.columns or gdf_hex_vis[var_col_name].isnull().all():
            print(f"  Saltando capa para '{var_col_name}', no disponible o todos los valores son nulos.")
            continue

        # Determinar si esta capa se muestra por defecto
        show_this_layer_by_default = (var_col_name == default_hex_variable_to_show)

        data_for_bins = gdf_hex_vis[var_col_name]
        try:
            if data_for_bins.nunique() > 1 and data_for_bins.max() > 0 :
                bins_val = sorted(list(set(data_for_bins.quantile([0, 0.05, 0.25, 0.5, 0.75, 0.95, 1]).tolist())))
                if len(bins_val) < 2: bins_val = 5
            else:
                bins_val = [0, data_for_bins.max()] if data_for_bins.max() > 0 else [0,1]
                if bins_val[0] == bins_val[1] : bins_val = [0, bins_val[1]*1.1 if bins_val[1] > 0 else 0.1]
                if len(bins_val) < 2 or bins_val[0] == bins_val[1]: bins_val = np.linspace(0, max(0.1, bins_val[1]), num=5).tolist()
            bins_val = sorted(list(set(el for el in bins_val if pd.notna(el) )))
            if not bins_val or len(bins_val) < 2: bins_val = 5
        except Exception as e_bin_calc:
            print(f"  Error calculando bins para {var_col_name}: {e_bin_calc}. Usando 5 bins por defecto.")
            bins_val = 5

        # Crear el Choropleth y AÑADIRLO DIRECTAMENTE AL MAPA
        choro_layer = folium.Choropleth(
            geo_data=gdf_hex_vis.to_json(), data=gdf_hex_vis,
            columns=['cell_id', var_col_name], key_on='feature.properties.cell_id',
            fill_color='BuPu', fill_opacity=0.75, line_opacity=0.3,
            legend_name=f'{layer_display_name} (R={RADIO_HEXAGONO_METROS}m)',
            bins=bins_val, highlight=True,
            name=layer_display_name, # Este nombre aparecerá en el LayerControl
            show=show_this_layer_by_default # Controla la visibilidad inicial
        )
        choro_layer.add_to(m_hex) # Añadir Choropleth directamente al mapa

        # Crear y añadir tooltips. Si se quiere que el tooltip esté asociado a esta capa
        # específica en el control de capas, se puede añadir a la misma capa Choropleth
        # o a un FeatureGroup que contenga solo esta capa Choropleth y sus tooltips.
        # Por simplicidad, podemos añadir un GeoJson para tooltips que se active con esta capa.

        tooltip_hex_fields_dynamic = ['cell_id', var_col_name, 'num_manzanas_en_hex', 'POBTOT_hex']
        tooltip_hex_aliases_dynamic = ['ID Hex:', f'{layer_display_name.split("(")[0].strip()}:', '# Manzanas:', 'Pob. Total Sum:']
        current_hex_flds = [f for f in tooltip_hex_fields_dynamic if f in gdf_hex_vis.columns]
        current_hex_als = [tooltip_hex_aliases_dynamic[tooltip_hex_fields_dynamic.index(f)] for f in current_hex_flds]

        if current_hex_flds:
            # El tooltip se puede añadir al objeto choro_layer si su clase base lo permite
            # o como un GeoJson separado que se activa con la capa.
            # Folium a veces es quisquilloso con esto.
            # Añadirlo al mapa directamente y controlar con el mismo 'name' puede funcionar.
             folium.features.GeoJson(
                data=gdf_hex_vis.to_json(), # Usar el mismo geo_data
                style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent', 'weight':0}, # Hacer invisible
                tooltip=folium.features.GeoJsonTooltip(
                    fields=current_hex_flds,
                    aliases=current_hex_als,
                    sticky=False,
                    localize=True,
                    style="background-color: white; color: black; font-size: 12px; padding: 5px;"
                ),
                name=f"Tooltips para {layer_display_name}", # Nombre diferente para que no se superponga en control
                show=show_this_layer_by_default # Mostrar si la capa principal se muestra
            ).add_to(m_hex) # Añadir al mapa principal

        print(f"  Capa hexagonal para '{layer_display_name}' generada y añadida al mapa.")
    # ***** FIN DEL CAMBIO PRINCIPAL *****

    folium.LayerControl(collapsed=False).add_to(m_hex)
    print("\nGenerando mapa interactivo de hexágonos...")
    display(m_hex)

except FileNotFoundError as e_file:
    print(f"  Error de Archivo: {e_file}")
except ValueError as e_val:
    print(f"  Error de Valor: {e_val}")
    import traceback; traceback.print_exc()
except ImportError as e_imp:
    print(f"  Error de Importación: {e_imp}. Asegúrate que 'geohexgrid' esté instalado.")
except duckdb.Error as e_duck:
    print(f"  Error de DuckDB: {e_duck}")
    import traceback; traceback.print_exc()
except Exception as e_general:
    print(f"  Ocurrió un error general: {e_general}")
    import traceback; traceback.print_exc()
finally:
    if con:
        try:
            con.close()
            print("\nConexión a DuckDB cerrada al finalizar.")
        except Exception as e_close:
            print(f"  Error al cerrar la conexión: {e_close}")

print("\n--- Fin del Script de Visualización Hexagonal (Nuevo Intento AssertionError) ---")

In [ ]:
# Celda 17: Guardar Agregación Hexagonal como GeoPackage
# ----------------------------------------------------

import geopandas as gpd
import pandas as pd # Asegurar que pandas esté importado si no lo está ya
import os

print("--- Iniciando Guardado de Agregación Hexagonal a GeoPackage ---")

# --- Variables de Configuración (Requeridas si esta celda se ejecuta de forma aislada) ---
# Asegúrate que estas coincidan con las celdas anteriores
CODIGO_ESTADO_STR = "01"
NOMBRE_ESTADO = "Aguascalientes" # Usado para el nombre de archivo
DIR_BASE_INEGI = "./inegi_data_ags" # Directorio donde se guardará el GeoPackage
# gdf_hex_vis debe existir de la Celda 16. Si no, necesitaríamos regenerarlo o leerlo.

# Nombre y ruta para el archivo GeoPackage de salida
GEOPACKAGE_OUTPUT_DIR = os.path.join(DIR_BASE_INEGI, "geopackages_output")
os.makedirs(GEOPACKAGE_OUTPUT_DIR, exist_ok=True) # Crear directorio si no existe
GEOPACKAGE_FILENAME = f"hexagonos_agregados_{NOMBRE_ESTADO.lower().replace(' ', '_')}_{CODIGO_ESTADO_STR}.gpkg"
GEOPACKAGE_FILE_PATH = os.path.join(GEOPACKAGE_OUTPUT_DIR, GEOPACKAGE_FILENAME)
LAYER_NAME_GPKG = f"hexagonos_ags_agregados_{RADIO_HEXAGONO_METROS}m" # Usar el radio en el nombre de la capa

# Verificar si gdf_hex_vis existe y tiene datos
if 'gdf_hex_vis' in locals() and isinstance(gdf_hex_vis, gpd.GeoDataFrame) and not gdf_hex_vis.empty:
    print(f"GeoDataFrame 'gdf_hex_vis' encontrado con {len(gdf_hex_vis)} hexágonos.")

    # Seleccionar las columnas que queremos guardar.
    # Es buena práctica ser explícito, aunque podríamos guardar todo gdf_hex_vis.
    # Columnas de interés: cell_id, geometry, y todas las que terminan en "_hex" o son "num_manzanas_en_hex"

    cols_to_keep = ['cell_id', 'geometry', 'num_manzanas_en_hex']
    variables_numericas_duck = ['POBTOT', 'VIVTOT', 'TVIVHAB', 'VPH_INTER', 'VPH_PC', 'VPH_CEL', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'GRAPROES']
    for var_base in variables_numericas_duck:
        col_hex = f"{var_base}_hex"
        if col_hex in gdf_hex_vis.columns:
            cols_to_keep.append(col_hex)

    # Crear una copia con solo las columnas deseadas para evitar modificar el GDF original
    gdf_to_save = gdf_hex_vis[cols_to_keep].copy()

    # Asegurar que la columna de geometría se llame 'geometry' (GeoPandas usualmente lo hace)
    if gdf_to_save.geometry.name != 'geometry':
        gdf_to_save.rename_geometry('geometry', inplace=True)

    print(f"Columnas a guardar en el GeoPackage: {gdf_to_save.columns.tolist()}")

    try:
        print(f"\nGuardando GeoDataFrame en: {GEOPACKAGE_FILE_PATH}")
        print(f"Nombre de la capa: {LAYER_NAME_GPKG}")

        # Guardar a GeoPackage
        # Si el archivo ya existe, geopandas lo sobrescribirá por defecto con to_file.
        # Si quieres diferentes capas en el mismo archivo GPKG, necesitarías 'mode="a"'
        # pero para una sola capa principal, el modo por defecto "w" (o si no existe lo crea) está bien.
        gdf_to_save.to_file(GEOPACKAGE_FILE_PATH, layer=LAYER_NAME_GPKG, driver="GPKG")

        print("\n¡GeoPackage guardado exitosamente!")
        print(f"  Ruta: {GEOPACKAGE_FILE_PATH}")
        print(f"  Capa: {LAYER_NAME_GPKG}")

        # Opcional: Descargar el archivo si estás en Google Colab
        # from google.colab import files
        # files.download(GEOPACKAGE_FILE_PATH)

    except Exception as e_save:
        print(f"  ¡ERROR al guardar el GeoDataFrame como GeoPackage!: {e_save}")
        import traceback
        traceback.print_exc()

else:
    print("¡ERROR! El GeoDataFrame 'gdf_hex_vis' no fue encontrado o está vacío.")
    print("Asegúrate de que la Celda 16 (o la celda que genera 'gdf_hex_vis') se haya ejecutado correctamente.")

print("\n--- Fin del Script de Guardado a GeoPackage ---")

In [ ]:
from google.colab import files
import os

files.download("./inegi_data_ags/geopackages_output/hexagonos_agregados_aguascalientes_01.gpkg")

In [ ]:
files.download("./inegi_data_ags/inegi_analisis_01.duckdb")